## This notebook is just for test.
## This cannot make a complete model because of memory error.
## Thank you.

In [ ]:

import numpy as np 
import pandas as pd 



# 1.前回データの確認

In [ ]:
path = "../input/eda-for-biginner-updated-to-english-ver"

In [ ]:
traindf = pd.read_csv(path+"/traindf.csv")
traindf

# おさらい

In [ ]:
tmp = traindf[traindf["landmark_id"]==7]
tmp

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
for a in tmp["path"]:
    img = cv2.imread(a)
    plt.figure()
    plt.imshow(img)

# おさらい2　import collectionを使って、各idの個数を数えた。それをcount数ごとに並べたのが、dfcnt

In [ ]:
dfcnt = pd.read_csv(path+"/dfcnt.csv")
dfcnt

In [ ]:
plt.scatter(dfcnt["id"],dfcnt["count"])

# この時点で、landmark idの種類は全部で81313個、最小枚数は2枚であることがわかる(前回は138982が6272個に注目してた)ので、
# 各landmark idごとに1枚訓練データ(traindata)、１枚検証データ(validation)にしてpytorchでモデルを作成することを考える。

In [ ]:
traindf

In [ ]:
dfcnt

## dfcntのidでfilteringして、一番上にきたやつをtrain data, 上から2つ目をvalidationとする
## わかりやすくするため、１個で説明

In [ ]:
tmp1 = dfcnt["id"].iloc[0]
tmp1

In [ ]:
tmpdf1 = traindf[traindf["landmark_id"]==tmp1]
tmpdf1

In [ ]:
tlist = []
vlist = []

In [ ]:
tlist.append(tmpdf1.iloc[0].values)
tlist

In [ ]:
vlist.append(tmpdf1.iloc[1].values)

In [ ]:
# これを繰り返す

In [ ]:
from tqdm import tqdm

In [ ]:
import os
os.path.exists("./tdf.csv")

In [ ]:
tlist = []
vlist = []

In [ ]:
if os.path.exists("./tdf.csv")==False:
    

    

    tmp1 = dfcnt["id"].values #.valuesでnumpy. for文はnumpyのほうが早いときがある。

    for a in tqdm(range(len(dfcnt))):

        tmpdf1 = traindf[traindf.landmark_id.values==tmp1[a]]
        tlist.append(tmpdf1.iloc[0].values)
        vlist.append(tmpdf1.iloc[1].values)

In [ ]:
tdf = pd.DataFrame(tlist,columns=tmpdf1.columns)
tdf["repair_id"]=np.arange(0,len(tdf),1)
tdf

In [ ]:
vdf = pd.DataFrame(vlist,columns=tmpdf1.columns)
vdf["repair_id"]=np.arange(0,len(vdf),1)
vdf

In [ ]:
if os.path.exists("./tdf.csv"):
    tdf = pd.read_csv("./tdf.csv")
    vdf = pd.read_csv("./vdf.csv")

In [ ]:
tdf.to_csv("tdf.csv",index=False)
vdf.to_csv("vdf.csv",index=False)

# 全部やっても良いが、自分で作成するときは10枚くらいでテストするほうが効率的

In [ ]:
tdf2 = tdf.iloc[:10,:]
vdf2 = vdf.iloc[:10,:]

In [ ]:
tdf2

In [ ]:
vdf2

# ここからpytorch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.nn import functional as F
from torchvision.models import resnet18
from albumentations import Normalize, Compose
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import os
import glob
import multiprocessing as mp



if torch.cuda.is_available():
    device = 'cuda:0'
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device = 'cpu'
print(f'Running on device: {device}')

# このサイトがとても分かりやすく書いてくれているので、迷ったらここを見る
https://qiita.com/takurooo/items/e4c91c5d78059f92e76d

# 1. transformの定義

In [ ]:
preprocess = Compose([
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1)
])

# resnextなどのpre-trainモデルは全て、同じ方法で正規化された入力画像を使用しなければならない。それの変換をこの関数で行う。値はdefault。
# Composeは今回あまり、意味をなさない
# https://betashort-lab.com/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9/albumentations%E3%81%AE%E3%81%BE%E3%81%A8%E3%82%81/ に詳細は書いてある

# 2. Dataset

In [ ]:
# 画像をどれだけ小さくするかの処理
ROWS = 32
COLS = 32

In [ ]:
class GLDataset(Dataset):
    
    def __init__(self,img_pass,labels,preprocess=None):
        self.img_pass = img_pass
        self.labels = labels
        self.preprocess = preprocess
        
    def __len__(self):
        return len(self.img_pass)
    
    def __getitem__(self,idx):
        
        # ここからdatasetに食わせる前の前処理の記述。
        
        img_pass = self.img_pass[idx]
        label = self.labels[idx]
        
        land = cv2.imread(img_pass)
        land = cv2.resize(land,(ROWS,COLS),interpolation = cv2.INTER_CUBIC)
        land = cv2.cvtColor(land,cv2.COLOR_BGR2RGB) # augmentを使うときにBGRからRGBにする必要があるのかもしれない。
        
        if self.preprocess is not None: # ここで、前処理を入れてnormalizationしている。
                augmented = self.preprocess(image=land) # preprocessのimageをfaceで読む
                land = augmented['image'] # https://betashort-lab.com/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9/albumentations%E3%81%AE%E3%81%BE%E3%81%A8%E3%82%81/　に書いてある
                
        return {'landmarks': land.transpose(2, 0, 1), 'label': np.array([label], dtype=int)}  # pytorchはchannnl, x, yの形。これは辞書型で返している。(扱いやすいというだけかも。)
        
        
        
        
        
        
        
        

## 1つ1つ追って、何やっているかを見ていく。

In [ ]:
land = cv2.imread(tdf2["path"].iloc[0])

In [ ]:
plt.imshow(land)

In [ ]:
land = cv2.resize(land,(ROWS,COLS),interpolation = cv2.INTER_CUBIC)

In [ ]:
plt.imshow(land)

In [ ]:
land = cv2.cvtColor(land,cv2.COLOR_BGR2RGB) # augmentを使うときにBGRからRGBにする必要があるのかもしれない。

In [ ]:
plt.imshow(land)

In [ ]:
augmented = preprocess(image=land) # preprocessのimageをfaceで読む
land = augmented['image']

In [ ]:
plt.imshow(land)

In [ ]:
land.shape

In [ ]:
land=land.transpose(2, 0, 1)

In [ ]:
land.shape

# Datasetのinstance化

In [ ]:
# instance化
train_dataset = GLDataset(
    img_pass=tdf2["path"],
    labels=tdf2["repair_id"].to_numpy(),
    preprocess=preprocess
)
#val_dataset = FaceValDataset(

val_dataset = GLDataset(
    img_pass=vdf2["path"],
    labels=vdf2["repair_id"].to_numpy(),
    preprocess=preprocess
)

In [ ]:
print(train_dataset[0])

# 3. DataLoader

In [ ]:
BATCH_SIZE = 2

#NUM_WORKERS = mp.cpu_count()
NUM_WORKERS = 0 # ここを0にしないと動かない。cpuの仕様個数。←実は動くことが判明。classの中身次第！

In [ ]:
#NUM_WORKERS = mp.cpu_count()
#NUM_WORKERS

In [ ]:
## DataLoaderはimport torch.utils.data.Datasetでimport済みのもの
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, #https://schemer1341.hatenablog.com/entry/2019/01/06/024605 を参考. idがわからなくなる
    num_workers=NUM_WORKERS
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

# 初日はここまでかな・・・